Correlation Matrix clustering

using hierarchical clustering
https://www.stat.cmu.edu/~cshalizi/350/lectures/08/lecture-08.pdf
algorithm outline:
a) each node is a single cluster
loop until the hard requirement fails
b) pair up nodes as a new cluster with different objective functions

In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib

Generate a correlation matrix of size 50*50

In [24]:
def genCorrelation(n):
    # generate a square correlation matrix with size n*n
    m = 2 * np.random.rand(n,n) - np.ones(n)
    for i in xrange(n):
        m[i][i] = 1
    for i in xrange(n):
        for j in xrange(i,n,1):
            m[i][j] = m[j][i]
    print('The ' + str(n) + '-th order correlation matrix is:')
    print(m)
    
    return m

#Basic requirement for clustering:
Correlation between any pair of stock in the same cluster must be higher than pair of stocks from different cluster. For example, A and B are from one cluster and C are for another cluster; Corr(A,B) >= Corr(A,C) and Corr(A,B) >= Corr(B,C)

In [79]:
M = genCorrelation(10)

The 10-th order correlation matrix is:
[[  1.00000000e+00   4.63709787e-01  -4.65073537e-01  -3.74248602e-02
    9.74214591e-01   5.51711134e-02   7.32787947e-01   9.72360346e-01
    8.97158447e-01  -6.86138765e-04]
 [  4.63709787e-01   1.00000000e+00   5.23264707e-01  -7.42171910e-01
   -6.57928959e-01  -7.26130306e-01   1.33230464e-01  -5.52759560e-01
    3.89229796e-01  -5.88148356e-01]
 [ -4.65073537e-01   5.23264707e-01   1.00000000e+00  -1.10733779e-01
    5.74685463e-01   9.63231605e-01  -2.12371552e-01  -3.27213922e-01
    6.83475680e-01   9.91262115e-01]
 [ -3.74248602e-02  -7.42171910e-01  -1.10733779e-01   1.00000000e+00
    5.98026922e-01  -3.26732766e-01  -7.81798943e-01   8.78368304e-01
   -4.23501509e-02  -6.04187752e-01]
 [  9.74214591e-01  -6.57928959e-01   5.74685463e-01   5.98026922e-01
    1.00000000e+00   1.76772814e-01   3.09591317e-01  -3.81218745e-01
   -5.71072393e-01   1.61180098e-01]
 [  5.51711134e-02  -7.26130306e-01   9.63231605e-01  -3.26732766e-01
    1.

In [75]:
def clustering(InitMatrix, m, clus):
    dim = m.shape[0]
    # find the largest 2 correlations in each row
    ind = np.zeros([dim,2])
    for i,irow in enumerate(m):
        ind[i] = np.argpartition(irow, -2)[-2:]
        
    # find the possible pair of clusters for combination
    feasiblePair = []
    for i in xrange(dim):
        for j in xrange(i+1,dim):
            # check which two clusters match
            if set(ind[i]) == set(ind[j]):
                feasiblePair.append(list(ind[i]))
    
    # find the possible path along this branch
    for ipair in feasiblePair[0:2]:
        iclus = clus[:]  # copy clus to iclus by value
        nrow = int(np.amin(ipair))   # put the new cluster in this row
        _row = int(np.amax(ipair))   # delete this row and corresponding column
        
        # update the new list for the indices for each cluster
        # assume clus = [[1,2,5],[3,4],[5,6,7,8],[9,10]], then combining 1st and 2nd
        # we have clus_new = [[1,2,3,4,5],[5,6,7,8],[9,10]]
        iclus[nrow] = iclus[nrow] + iclus[_row]
        del iclus[_row]
        
        # construct the new matrix by combining two clusters
        nmatrix = np.delete(np.delete(m,_row,0), _row, 1) # get a dim-1 dimensional matrix
        print len(iclus)
        for i in xrange(dim-1):
            if i == nrow:
                # compute the minimum correlation
                # get the submatrix with indices of elements in the new cluster
                _matrix = InitMatrix[np.ix_(iclus[nrow], iclus[nrow])]
                # compute the minimum based on the original matrix
                nmatrix[i][i] = np.amin(_matrix)
            else:
                # compute the maximum correlation between clusters
                # get the indices of the lements of i cluster
                
                _r = iclus[nrow] + iclus[i]
                _matrix = InitMatrix[np.ix_(_r, _r)]
                # compute the maximum based on the original matrix 
                nmatrix[nrow][i] = np.amax(_matrix)
                nmatrix[i][nrow] = nmatrix[nrow][i]
        print ipair
        print iclus
        print nmatrix
    

In [80]:
clus = []
for i in xrange(10):
    clus.append([i])
clustering(M,M,clus)

look at here
10
9
[4.0, 0.0]
[[0, 4], [1], [2], [3], [5], [6], [7], [8], [9]]
[[ 0.97421459  1.          1.          1.          1.          1.          1.
   1.          1.        ]
 [ 1.          1.          0.52326471 -0.74217191 -0.72613031  0.13323046
  -0.55275956  0.3892298  -0.58814836]
 [ 1.          0.52326471  1.         -0.11073378  0.9632316  -0.21237155
  -0.32721392  0.68347568  0.99126212]
 [ 1.         -0.74217191 -0.11073378  1.         -0.32673277 -0.78179894
   0.8783683  -0.04235015 -0.60418775]
 [ 1.         -0.72613031  0.9632316  -0.32673277  1.          0.56435891
   0.23960962  0.4819833  -0.76737067]
 [ 1.          0.13323046 -0.21237155 -0.78179894  0.56435891  1.
   0.53534765 -0.27086105  0.73704678]
 [ 1.         -0.55275956 -0.32721392  0.8783683   0.23960962  0.53534765
   1.         -0.12680115 -0.35409109]
 [ 1.          0.3892298   0.68347568 -0.04235015  0.4819833  -0.27086105
  -0.12680115  1.          0.00405176]
 [ 1.         -0.58814836  0.99126

In [62]:
aa =[[2,3],[6,0,3,3],[6,7]]
print aa[0:2]

[[2, 3], [6, 0, 3, 3]]


In [ ]:
def clusteringMaxSumCorrelation():
    '''
    Summation over Correlations between stocks in the same cluster is maximized. 
    This sum of correlation is sum of all the elements in correlation matrix. I.e. 
    if you constructed a cluster with 100 stocks. You can calculate 100 x 100 correlation
    matrix and sum up all the elements to get "sum of correlation". In other word, 
    stocks in the same cluster must be highly correlated.
    '''
    pass

In [ ]:
def clusteringUniformSize():
    '''
    The number of clusters is not restricted as long as you have more than four 
    clusters. But standard deviation of the numbers of clusters is to be minimized. 
    For example, clusters of 100 stocks, 100 stocks, 100 stocks and 200 stocks are 
    better than those of 300 stocks, 160 stocks, 35 stocks and 5 stocks.
    '''
    
    # each time we merge two clusters, the requirements are
    # 1) the basic requirement must be satisfied
    # 2) the resulting pattern of merging has the best objective function
    
    
    for i in range(n):
        for j in range(n):
            # merge cluster i and cluster j
            # if resulting a qualified matrix, then compute its objective function
            if checkFeasibility(matrix):
                calcObjUniformSize()
    
    pass

In [ ]:
def clusteringIndependentReturn():
    '''
    You can calculate the average of returns of stocks in the same cluster. 
    If you have four clusters, you will have four average returns, from which 4x4 
    correlation matrix can be calculated. Summation over Correlation  between average 
    returns of clusters is minimized. In other word, clusters must not be correlated in 
    terms of average return
    '''
    pass

In [62]:
a = np.array([9, 9, 4, 3, 9, 9, 0, 4, 6, 0])
ind = np.argpartition(a, -2)[-2:]
ind = set(ind)
print ind
print ind==set([4,5])

set([4, 5])
True
